<a href="https://colab.research.google.com/github/Becca8463/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
application_copy1 = application_df.copy()

In [5]:
application_copy2 = application_df.copy()

In [6]:
application_copy3 = application_df.copy()

In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_copy1= application_df.drop(columns=['EIN', 'NAME'])

In [8]:
# Determine the number of unique values in each column.
unique_value_counts = application_copy1.nunique()
unique_value_counts

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [9]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_value_counts = application_copy1['APPLICATION_TYPE'].value_counts()
application_value_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 10
app_types_to_replace = application_value_counts[application_value_counts < app_cutoff].index

# Replace in dataframe
for app in app_types_to_replace:
    application_copy1['APPLICATION_TYPE'] = application_copy1['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_copy1['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_copy1 = pd.get_dummies(application_copy1)

In [12]:
# Split our preprocessed data into our features and target arrays
X = application_copy1.drop(columns = 'IS_SUCCESSFUL').values
y = application_copy1['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 50)                  │           5,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           1,020 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 25)                  │             525 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,221 (28.21 KB)

 Trainable params: 7,221 (28.21 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6882 - loss: 0.6102
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7307 - loss: 0.5512
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7312 - loss: 0.5491
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7300 - loss: 0.5516
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7327 - loss: 0.5470
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7331 - loss: 0.5434
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7369 - loss: 0.5390
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7359 - loss: 0.5416
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7409 - loss: 0.5355
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7424 - loss: 0.5371
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7335 - loss: 0.5392
Epoch 12/100
858/858 ━━━━━━━━━━━━━━━━━━━━

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 2ms/step - accuracy: 0.7289 - loss: 0.5640
Loss: 0.5640071630477905, Accuracy: 0.728863000869751


In [18]:
#dropping only EIN column

application_copy2 = application_df.copy()

# Drop the non-beneficial ID columns, 'EIN'
application_copy2 = application_df.drop(columns=['EIN'])

In [19]:
# Determine the number of unique values in each column.
unique_value_counts = application_copy2.nunique()
unique_value_counts

,0
NAME,19568
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747


In [20]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_value_counts = application_copy2['APPLICATION_TYPE'].value_counts()
application_value_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [21]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 800
app_types_to_replace = application_value_counts[application_value_counts < app_cutoff].index

# Replace in dataframe
for app in app_types_to_replace:
    application_copy2['APPLICATION_TYPE'] = application_copy2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_copy2['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
Other,2266
T4,1542
T6,1216
T5,1173
T19,1065


In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
application_copy2 = pd.get_dummies(application_copy2)

In [23]:
# Split our preprocessed data into our features and target arrays
X = application_copy1.drop(columns = 'IS_SUCCESSFUL').values
y = application_copy1['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 8)                   │             904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │              45 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 955 (3.73 KB)

 Trainable params: 955 (3.73 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6509 - loss: 0.6508
Epoch 2/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7300 - loss: 0.5620
Epoch 3/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7323 - loss: 0.5545
Epoch 4/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7283 - loss: 0.5540
Epoch 5/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7318 - loss: 0.5486
Epoch 6/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7304 - loss: 0.5499
Epoch 7/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7322 - loss: 0.5506
Epoch 8/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7321 - loss: 0.5462
Epoch 9/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7348 - loss: 0.5448
Epoch 10/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7342 - loss: 0.5429
Epoch 11/100
751/751 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7293 - loss: 0.5495
Epoch 12/100
751/751 ━━━━━━━━━━━━━━━━━━━━

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 0s - 1ms/step - accuracy: 0.7268 - loss: 0.5563
Loss: 0.5562835335731506, Accuracy: 0.7268221378326416


In [29]:
#add more nuerons to a hidden layer

application_copy3 = application_df.copy()

In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_copy3 = application_df.drop(columns=['EIN', 'NAME'])

In [31]:
# Determine the number of unique values in each column.
unique_value_counts = application_copy3.nunique()
unique_value_counts

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [32]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_value_counts = application_copy3['APPLICATION_TYPE'].value_counts()
application_value_counts


,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [35]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff = 750
app_types_to_replace = application_value_counts[application_value_counts < app_cutoff].index

# Replace in dataframe
for app in app_types_to_replace:
    application_copy3['APPLICATION_TYPE'] = application_copy3['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_copy3['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
Other,2266
T4,1542
T6,1216
T5,1173
T19,1065


In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
application_copy3 = pd.get_dummies(application_copy3)

In [38]:
# Split our preprocessed data into our features and target arrays
X = application_copy3.drop('IS_SUCCESSFUL', axis=1).values
y = application_copy3['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=76)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 120)                 │          12,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 100)                 │          12,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,921 (97.35 KB)

 Trainable params: 24,921 (97.35 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [42]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7154 - loss: 0.5917
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7330 - loss: 0.5571
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7284 - loss: 0.5537
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7326 - loss: 0.5485
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7323 - loss: 0.5482
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7369 - loss: 0.5447
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7369 - loss: 0.5449
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7357 - loss: 0.5440
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7327 - loss: 0.5451
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7300 - loss: 0.5469
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7380 - loss: 0.5424
Epoch 12/100
858/858 ━━━━━━━━━━━━━━━━━━━━

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 2ms/step - accuracy: 0.7296 - loss: 0.5774
Loss: 0.5773962140083313, Accuracy: 0.7295918464660645


In [45]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')